In [2]:
!pip3 install --upgrade rouge-score bert-score nltk -q
!pip install contractions -q
!pip install evaluate -q
import nltk
nltk.download("punkt")
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('wordnet2022')

! cp -rf /usr/share/nltk_data/corpora/wordnet2022 /usr/share/nltk_data/corpora/wordnet # fix for kaggle (https://www.kaggle.com/code/sharanharsoor/tutorial-word-sense-disambiguation)

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 23.2 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.9.1 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.9 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet2022 to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/wordnet2022.zip.


In [ ]:
import re
import pandas as pd
import numpy as np
import contractions
from sklearn.model_selection import train_test_split
import json
import torch
import pandas as pd
import evaluate
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq
from rouge_score import rouge_scorer
from bert_score import score

## Preprocessing

In [4]:
with open("/kaggle/input/nlp-a3-q2/abbreviation.json") as f:
    abbreviations = json.load(f)

url_pattern = re.compile(r"http\S+|www\S+|https\S+")
special_pattern = re.compile(r"[^a-zA-Z0-9.,!?'\s]")
abbreviation_regex = re.compile(
    r'(?<=\s)(' + '|'.join(map(re.escape, abbreviations.keys())) + r')(?=\s)',
    re.IGNORECASE
)

def preprocess_data(text):
    text = text.lower() # Convert to lowercase
    text = contractions.fix(text) # Expand contractions

    text = abbreviation_regex.sub(lambda m: abbreviations[m.group(0).lower()], text) # Expand abbreviations

    text = url_pattern.sub("", text) # Remove URLs
    text = special_pattern.sub("", text) # Remove special characters

    text_words = text.strip().split()
    while "" in text_words:
        text_words.remove("")
    text = " ".join(text_words) # Remove extra spaces

    return text

## Model Training

In [ ]:
def evaluate_model(trainer, tokenizer, test_dataset, batch_size=4):
    predictions = []
    all_labels = []

    for i in range(0, len(test_dataset), batch_size):
        end_idx = min(i + batch_size, len(test_dataset))
        batch = test_dataset.select(range(i, end_idx))
        
        with torch.no_grad():
            prediction_outputs = trainer.predict(batch)
        
        pred_ids = prediction_outputs.predictions
        label_ids = prediction_outputs.label_ids

        if isinstance(pred_ids, tuple):
            pred_ids = pred_ids[0]
        
        if len(pred_ids.shape) == 3:
            pred_ids = np.argmax(pred_ids, axis=-1)
        
        label_ids[label_ids == -100] = tokenizer.pad_token_id
        
        preds = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
        batch_labels = tokenizer.batch_decode(label_ids, skip_special_tokens=True)
        
        predictions.extend(preds)
        all_labels.extend(batch_labels)
        
        del prediction_outputs, pred_ids, label_ids
        torch.cuda.empty_cache()

    raw_labels = [[label] for label in all_labels]

    rouge = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    rougeL_precision = 0.0
    rougeL_recall = 0.0
    rougeL_f1 = 0.0

    for pred, label in zip(predictions, all_labels):
        scores = rouge.score(label, pred)
        rougeL_precision += scores["rougeL"].precision
        rougeL_recall += scores["rougeL"].recall
        rougeL_f1 += scores["rougeL"].fmeasure

    rougeL_precision /= len(predictions)
    rougeL_recall /= len(predictions)
    rougeL_f1 /= len(predictions)

    bleu_metric = evaluate.load("bleu")
    bleu4 = bleu_metric.compute(predictions=predictions, references=raw_labels, max_order=4)

    P, R, F1 = score(predictions, all_labels, lang="en", verbose=False)
    bert_precision = P.mean().item()
    bert_recall = R.mean().item()
    bert_f1 = F1.mean().item()


    print(f"ROUGE-L Precision: {rougeL_precision:.4f}")
    print(f"ROUGE-L Recall: {rougeL_recall:.4f}")
    print(f"ROUGE-L F1: {rougeL_f1:.4f}")
    print(f"BLEU-4: {bleu4['bleu']:.4f}")
    print(f"BERTscore Precision: {bert_precision:.4f}")
    print(f"BERTscore Recall: {bert_recall:.4f}")
    print(f"BERTscore F1: {bert_f1:.4f}")

    return rougeL_precision, rougeL_recall, rougeL_f1, bleu4, bert_precision, bert_recall, bert_f1


def train_model(train_df, val_df, test_df, model_type, inp_max_length=384, out_max_length=128, epochs=10):
    if model_type == "bart":
        model_name = "facebook/bart-base"
    elif model_type == "t5":
        model_name = "google-t5/t5-small"
    else:
        raise ValueError("Invalid model type. Choose 'bart' or 't5'.")
    
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

    train_data = tokenizer(train_df["Processed Text"].to_list(), max_length=512, padding="max_length", truncation=True, return_tensors="pt")
    train_data["labels"] = tokenizer(train_df["Normalized Claim"].to_list(), max_length=128, padding="max_length", truncation=True, return_tensors="pt")["input_ids"]

    val_data = tokenizer(val_df["Processed Text"].to_list(), max_length=512, padding="max_length", truncation=True, return_tensors="pt")
    val_data["labels"] = tokenizer(val_df["Normalized Claim"].to_list(), max_length=128, padding="max_length", truncation=True, return_tensors="pt")["input_ids"]

    test_data = tokenizer(test_df["Processed Text"].to_list(), max_length=512, padding="max_length", truncation=True, return_tensors="pt")
    test_data["labels"] = tokenizer(test_df["Normalized Claim"].to_list(), max_length=128, padding="max_length", truncation=True, return_tensors="pt")["input_ids"]

    train_dataset = Dataset.from_dict(train_data)
    val_dataset = Dataset.from_dict(val_data)
    test_dataset = Dataset.from_dict(test_data)

    training_args = TrainingArguments(
        output_dir=f"./results/{model_type}",
        eval_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=4,
        num_train_epochs=epochs,
        weight_decay=0.01,
        logging_steps=50,
        load_best_model_at_end=True,
        save_strategy="epoch",
        save_total_limit=1,
        logging_dir='./logs',
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=data_collator,
    )

    trainer.train()

    trainer.save_model(f"./models/{model_type}")

    evaluate_model(trainer, tokenizer, test_dataset)

    return model, trainer, tokenizer

In [6]:
df = pd.read_csv("/kaggle/input/nlp-a3-q2/CLAN_data.csv")
df["Processed Text"] = df["Social Media Post"].apply(preprocess_data)

train_df, val_test_df = train_test_split(df, test_size=0.3)
val_df, test_df = train_test_split(val_test_df, test_size=0.5)

print(f"Number of training samples: {len(train_df)}")
print(f"Number of validation samples: {len(val_df)}")
print(f"Number of test samples: {len(test_df)}")

Number of training samples: 1967
Number of validation samples: 422
Number of test samples: 422


## BART

In [7]:
model, trainer, tokenizer = train_model(train_df, val_df, test_df, model_type="bart", inp_max_length=512, out_max_length=128, epochs=10)

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ROUGE-L Precision: 0.5153
ROUGE-L Recall: 0.5095
ROUGE-L F1: 0.5116
BLEU-4: 0.2975
BERTscore Precision: 0.8941
BERTscore Recall: 0.9033
BERTscore F1: 0.8986


In [8]:
wandb.finish()

eval/loss,█▁▁▁▁▁▁▁▁▁
eval/runtime,▂▃█▂▂▁▂▁▁▂
eval/samples_per_second,▆▆▁▇▇█▇██▇
eval/steps_per_second,▆▆▁▇▇█▇██▇
test/loss,▃▅▄▃▂▁▃▂▃▂▂▆█▃▃▃▄▂▁▂▆▃▄▁▄▂▃█▂▂▄▅▃▃▃▂▃▃▄▄
test/runtime,▄▄▄▅▄▃▄▃▄▄▃▃▃▂▃▃▂▄▆█▃▂▃▁▂▂▁▂▁▄▄▅▃▃▄█▇▄▂▅
test/samples_per_second,▃▆▇▆▇▇█▇▇▇██▇█████████▇▆███▇██▇▇▇▆▇████▁
test/steps_per_second,▁▅▂▆▆▆▆▆▅▇▆▅▇▅▆▆▆▆▆█▆▃▅▆▃▇▆████▅▇▆▅▆▆▂▇▅
train/epoch,▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▂▂▂▃▄▅▅▅▆█████████████████████████████
train/grad_norm,█▇▅▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


In [23]:
!zip -r bart_model.zip /kaggle/working/models/bart

  adding: kaggle/working/models/bart/ (stored 0%)
  adding: kaggle/working/models/bart/generation_config.json (deflated 47%)
  adding: kaggle/working/models/bart/model.safetensors (deflated 8%)
  adding: kaggle/working/models/bart/config.json (deflated 63%)
  adding: kaggle/working/models/bart/training_args.bin (deflated 51%)


## T5

In [10]:
model, trainer, tokenizer = train_model(train_df, val_df, test_df, model_type="t5", inp_max_length=512, out_max_length=128, epochs=10)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

...

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,4.316400,1.237726
2,1.435400,0.909628
3,0.925900,0.817430
4,0.838900,0.765380
5,0.777100,0.733223
6,0.729400,0.713044
7,0.714400,0.697891
8,0.726300,0.689501
9,0.712000,0.684447
10,0.711300,0.683487


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked t

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ROUGE-L Precision: 0.4056
ROUGE-L Recall: 0.3592
ROUGE-L F1: 0.3793
BLEU-4: 0.1641
BERTscore Precision: 0.8461
BERTscore Recall: 0.8598
BERTscore F1: 0.8528


In [11]:
wandb.finish()

eval/loss,█▄▃▂▂▁▁▁▁▁
eval/runtime,▇█▅▃▂▆▄▄▂▁
eval/samples_per_second,▂▁▄▆▇▃▅▅▇█
eval/steps_per_second,▂▁▄▆▇▃▅▅▇█
test/loss,▃▃▃▄▂▃▂▂▄▃▂▆▂▂▅▁▂▄▂▃▃▃▁▂▃▄▄▆▂█▁▂▂▂▁▆▂▅▂▁
test/runtime,▂▂▂▂▂▂▂▂▂▂▂▂▂█▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁
test/samples_per_second,█▇█████████▇█▇▇▇▁▇██▇▇▇▇██▇█▇▇█████▇████
test/steps_per_second,▄▅▅▅▅▅▅▅▅▅▅▅▅▅▄▅▄▄▁▅▄▄▄▅▅▅▅▄▅▅▅▅▅▅▅▅▅▅▅█
train/epoch,▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▂▂▃▃▄▆▆▇██████████████████████████████
train/grad_norm,█▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


In [24]:
!zip -r t5_model.zip /kaggle/working/models/t5

  adding: kaggle/working/models/t5/ (stored 0%)
  adding: kaggle/working/models/t5/generation_config.json (deflated 29%)
  adding: kaggle/working/models/t5/model.safetensors (deflated 10%)
  adding: kaggle/working/models/t5/config.json (deflated 62%)
  adding: kaggle/working/models/t5/training_args.bin (deflated 52%)


## Inference

In [16]:
all_data = pd.read_csv("/kaggle/input/nlp-a3-q2/CLAN_data.csv")
all_data.iloc[:100].to_csv("/kaggle/working/test_data.csv")

In [25]:
def inference(test_csv_path, model_checkpoint_path, model_type, inp_max_length=384, out_max_length=128, batch_size=4):
    if model_type == "bart":
        model_name = "facebook/bart-base"
    elif model_type == "t5":
        model_name = "google-t5/t5-small"
    else:
        raise ValueError("Invalid model type. Choose 'bart' or 't5'.")
    
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint_path)

    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

    test_df = pd.read_csv(test_csv_path)
    test_df["Processed Text"] = test_df["Social Media Post"].apply(preprocess_data)

    test_data = tokenizer(test_df["Processed Text"].to_list(), max_length=inp_max_length, padding="max_length", truncation=True, return_tensors="pt")
    test_data["labels"] = tokenizer(test_df["Normalized Claim"].to_list(), max_length=out_max_length, padding="max_length", truncation=True, return_tensors="pt")["input_ids"]

    test_dataset = Dataset.from_dict(test_data)

    trainer = Trainer(
        model=model,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

    evaluate_model(trainer, tokenizer, test_dataset, batch_size=batch_size)

    model.eval()
    generated_claims = []
    for i in range(0, len(test_df), batch_size):
        end_idx = min(i + batch_size, len(test_df))
        inputs = tokenizer(test_df["Processed Text"].iloc[i:end_idx].to_list(), 
                           return_tensors="pt", 
                           padding=True, 
                           truncation=True, 
                           max_length=inp_max_length).to(model.device)
        
        with torch.no_grad():
            outputs = model.generate(**inputs, max_length=out_max_length)
        
        preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        generated_claims.extend(preds)

    test_df["Generated Claim"] = generated_claims
    test_df.to_csv("test_predictions.csv", index=False)

    return test_df

In [26]:
inference("/kaggle/working/test_data.csv", "/kaggle/working/models/bart", "bart", inp_max_length=384, out_max_length=128, batch_size=4)

<ipython-input-25-c5e6a050abc2>:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ROUGE-L Precision: 0.6217
ROUGE-L Recall: 0.6175
ROUGE-L F1: 0.6188
BLEU-4: 0.3930
BERTscore Precision: 0.9178
BERTscore Recall: 0.9249
BERTscore F1: 0.9213


,Unnamed: 0,PID,Social Media Post,Normalized Claim,Processed Text,Generated Claim
0,0,1,"President \n @realDonaldTrump\n : ""Biden's pla...",Biden’s energy plan would get rid of seniors’ ...,president realdonaldtrump biden's planning wou...,“President Donald Trump’s plan would mean Amer...
1,1,2,IMPORTANT ANNOUNCEMENT - CORONAVIRUS\nLast eve...,"If someone with the new coronavirus sneezes, i...",important announcement coronavirus last evenin...,A social media post claims that drinking warm ...
2,2,3,Heart is delighted to hear,Heart is delighted to hear,heart is delighted two hear,Heart is delighted to hear
3,3,4,An allowed appeal is one where the initial ref...,The vast majority of people coming across the ...,an allowed appeal is one where these initial r...,The Supreme Court has ruled that the governmen...
4,4,5,WARM WATER THERAPY\nDr. D. Mensah Asare says t...,A widely popular social media post claims that...,warm water therapy drive d. mensah asare says ...,Warm water can treat various diseases
...,...,...,...,...,...,...
95,95,96,I want you to truly understand that currently ...,PCR tests for Covid-19 have a 100% false posit...,i want you two truly understand that currently...,The Covid-19 vaccine has a 100% false positive...
96,96,97,Rest In Peace Mino.,“Italian football agent Mino Raiola has died a...,rest into peace mino.,President Bongbong Marcos has ordered the mili...
97,97,98,You science IS politics.\n\nAll the RCTs show ...,masks are ineffective and vaccines unnecessary...,you science is politics. all these rcts show n...,Vaccines are ineffective for most people
98,98,99,U.S. Health and Human Services Director Xavier...,After the U.S. Health and Human Services Direc...,you.s. health and human services director xavi...,“You.S. Health and Human Services Director Xav...
